# YOLO Box Detection Training Notebook

This notebook provides a comprehensive guide for training a YOLO model for box detection and counting using Roboflow datasets.

## Overview
- **Objective**: Train a YOLO model to detect and count boxes in images
- **Framework**: Ultralytics YOLO v8
- **Data Source**: Roboflow datasets
- **Output**: Trained model for box detection and counting

## Workflow
1. Install and import required libraries
2. Set up Roboflow API connection
3. Download and prepare dataset
4. Configure YOLO model
5. Train the model
6. Test on sample images
7. Implement box counting logic
8. Evaluate performance
9. Deploy for real-time inference

## 1. Install and Import Required Libraries

First, let's install and import all the necessary libraries for YOLO model training and box detection.

In [ ]:
# Install required packages
!pip install ultralytics roboflow opencv-python matplotlib seaborn pandas numpy pillow pyyaml

# Verify PyTorch installation
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Import required libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yaml
from pathlib import Path
import json
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Import YOLO and Roboflow
from ultralytics import YOLO
from roboflow import Roboflow

# Set matplotlib style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("All libraries imported successfully!")

## 2. Set Up Roboflow API Connection

Configure your Roboflow API credentials to access datasets and model management features.

In [ ]:
# Roboflow Configuration
# Get your API key from: https://roboflow.com/

# Option 1: Set your credentials here (not recommended for production)
ROBOFLOW_API_KEY = "your_api_key_here"  # Replace with your actual API key
WORKSPACE_NAME = "your_workspace"       # Replace with your workspace name
PROJECT_NAME = "box-detection"          # Replace with your project name
VERSION = 1                             # Dataset version

# Option 2: Load from environment variables (recommended)
# import os
# ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")
# WORKSPACE_NAME = os.getenv("ROBOFLOW_WORKSPACE")
# PROJECT_NAME = os.getenv("ROBOFLOW_PROJECT", "box-detection")

# Initialize Roboflow client
try:
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    workspace = rf.workspace(WORKSPACE_NAME)
    project = workspace.project(PROJECT_NAME)
    print(f"✅ Connected to Roboflow project: {WORKSPACE_NAME}/{PROJECT_NAME}")
except Exception as e:
    print(f"❌ Error connecting to Roboflow: {e}")
    print("Please check your API key and project details")

## 3. Download and Prepare Dataset

Download the box detection dataset from Roboflow and prepare it for YOLO training.

In [ ]:
# Download dataset from Roboflow
print("📥 Downloading dataset...")

try:
    # Get dataset version
    version = project.version(VERSION)
    
    # Download in YOLO format
    dataset = version.download("yolov8", location="./datasets")
    
    print(f"✅ Dataset downloaded successfully!")
    print(f"📁 Dataset location: {dataset.location}")
    
    # Store dataset path for later use
    DATASET_PATH = dataset.location
    
except Exception as e:
    print(f"❌ Error downloading dataset: {e}")
    # Fallback to manual path if download fails
    DATASET_PATH = "./datasets/box-detection-1"

In [ ]:
# Explore dataset structure
print("🔍 Exploring dataset structure...")

# List dataset contents
if os.path.exists(DATASET_PATH):
    for item in os.listdir(DATASET_PATH):
        item_path = os.path.join(DATASET_PATH, item)
        if os.path.isdir(item_path):
            count = len([f for f in os.listdir(item_path) if f.endswith(('.jpg', '.png', '.txt'))])
            print(f"📂 {item}: {count} files")
        else:
            print(f"📄 {item}")
    
    # Load data.yaml to check configuration
    yaml_path = os.path.join(DATASET_PATH, "data.yaml")
    if os.path.exists(yaml_path):
        with open(yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)
        
        print("\n📋 Dataset Configuration:")
        print(f"  Classes: {data_config.get('nc', 'Unknown')}")
        print(f"  Class names: {data_config.get('names', 'Unknown')}")
        print(f"  Train path: {data_config.get('train', 'Unknown')}")
        print(f"  Validation path: {data_config.get('val', 'Unknown')}")
        
        # Store for later use
        NUM_CLASSES = data_config.get('nc', 1)
        CLASS_NAMES = data_config.get('names', ['box'])
    else:
        print("⚠️ data.yaml not found")
        NUM_CLASSES = 1
        CLASS_NAMES = ['box']
else:
    print("❌ Dataset path not found")

In [ ]:
# Visualize sample images from the dataset
def visualize_sample_images(dataset_path, num_samples=6):
    """Visualize sample images with their annotations"""
    
    train_images_path = os.path.join(dataset_path, "train", "images")
    train_labels_path = os.path.join(dataset_path, "train", "labels")
    
    if not os.path.exists(train_images_path):
        print("❌ Training images not found")
        return
    
    # Get sample images
    image_files = [f for f in os.listdir(train_images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    sample_files = image_files[:num_samples]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, img_file in enumerate(sample_files):
        if i >= num_samples:
            break
            
        # Load image
        img_path = os.path.join(train_images_path, img_file)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load corresponding label
        label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt')
        label_path = os.path.join(train_labels_path, label_file)
        
        # Draw bounding boxes if label exists
        if os.path.exists(label_path):
            h, w = image.shape[:2]
            with open(label_path, 'r') as f:
                lines = f.readlines()
            
            box_count = 0
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    _, x_center, y_center, width, height = map(float, parts[:5])
                    
                    # Convert YOLO format to pixel coordinates
                    x1 = int((x_center - width/2) * w)
                    y1 = int((y_center - height/2) * h)
                    x2 = int((x_center + width/2) * w)
                    y2 = int((y_center + height/2) * h)
                    
                    # Draw bounding box
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    box_count += 1
            
            title = f"{img_file}\n{box_count} boxes"
        else:
            title = f"{img_file}\nNo labels"
        
        axes[i].imshow(image)
        axes[i].set_title(title, fontsize=10)
        axes[i].axis('off')
    
    # Hide unused subplots
    for i in range(len(sample_files), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle("Sample Training Images with Box Annotations", fontsize=16, y=1.02)
    plt.show()

# Visualize samples
visualize_sample_images(DATASET_PATH)

## 4. Configure YOLO Model

Set up the YOLO model configuration including architecture selection and training parameters.

In [ ]:
# YOLO Model Configuration
MODEL_SIZE = "yolov8n"  # Options: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
                        # n=nano (fastest), s=small, m=medium, l=large, x=extra large (most accurate)

# Training parameters
TRAINING_CONFIG = {
    'epochs': 100,           # Number of training epochs
    'batch': 16,            # Batch size (adjust based on GPU memory)
    'imgsz': 640,           # Image size for training
    'patience': 50,         # Early stopping patience
    'save_period': 10,      # Save model every N epochs
    'workers': 8,           # Number of dataloader workers
    'optimizer': 'auto',    # Optimizer (auto, SGD, Adam, AdamW, etc.)
    'lr0': 0.01,           # Initial learning rate
    'lrf': 0.01,           # Final learning rate (lr0 * lrf)
    'momentum': 0.937,      # SGD momentum/Adam beta1
    'weight_decay': 0.0005, # Optimizer weight decay
    'warmup_epochs': 3,     # Warmup epochs
    'warmup_momentum': 0.8, # Warmup initial momentum
    'box': 7.5,            # Box loss gain
    'cls': 0.5,            # Class loss gain
    'dfl': 1.5,            # DFL loss gain
    'pose': 12.0,          # Pose loss gain (pose models only)
    'kobj': 2.0,           # Keypoint obj loss gain (pose models only)
    'label_smoothing': 0.0, # Label smoothing (fraction)
    'nbs': 64,             # Nominal batch size
    'hsv_h': 0.015,        # Image HSV-Hue augmentation (fraction)
    'hsv_s': 0.7,          # Image HSV-Saturation augmentation (fraction)
    'hsv_v': 0.4,          # Image HSV-Value augmentation (fraction)
    'degrees': 0.0,        # Image rotation (+/- deg)
    'translate': 0.1,      # Image translation (+/- fraction)
    'scale': 0.5,          # Image scale (+/- gain)
    'shear': 0.0,          # Image shear (+/- deg)
    'perspective': 0.0,    # Image perspective (+/- fraction), range 0-0.001
    'flipud': 0.0,         # Image flip up-down (probability)
    'fliplr': 0.5,         # Image flip left-right (probability)
    'mosaic': 1.0,         # Image mosaic (probability)
    'mixup': 0.0,          # Image mixup (probability)
    'copy_paste': 0.0,     # Segment copy-paste (probability)
}

print("🔧 Model Configuration:")
print(f"  Model: {MODEL_SIZE}")
print(f"  Epochs: {TRAINING_CONFIG['epochs']}")
print(f"  Batch size: {TRAINING_CONFIG['batch']}")
print(f"  Image size: {TRAINING_CONFIG['imgsz']}")
print(f"  Learning rate: {TRAINING_CONFIG['lr0']}")
print(f"  Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# Initialize model
model = YOLO(f'{MODEL_SIZE}.pt')  # Load pretrained model
print(f"✅ Loaded {MODEL_SIZE} model")

## 5. Train the YOLO Model

Start training the YOLO model on the box detection dataset.

In [ ]:
# Start training
print("🚀 Starting model training...")
print("This may take several hours depending on your dataset size and hardware.")

try:
    # Train the model
    results = model.train(
        data=os.path.join(DATASET_PATH, "data.yaml"),
        **TRAINING_CONFIG
    )
    
    print("✅ Training completed successfully!")
    print(f"📁 Results saved to: {results.save_dir}")
    
    # Store the best model path
    BEST_MODEL_PATH = os.path.join(results.save_dir, "weights", "best.pt")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    # Set a fallback path
    BEST_MODEL_PATH = f"runs/detect/train/weights/best.pt"

In [ ]:
# Visualize training results
def plot_training_results(results_dir):
    """Plot training metrics from results"""
    
    results_csv = os.path.join(results_dir, "results.csv")
    
    if os.path.exists(results_csv):
        # Load training results
        df = pd.read_csv(results_csv)
        df.columns = df.columns.str.strip()  # Remove any whitespace
        
        # Create subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot loss curves
        if 'train/box_loss' in df.columns:
            axes[0, 0].plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', color='blue')
            if 'val/box_loss' in df.columns:
                axes[0, 0].plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', color='red')
            axes[0, 0].set_title('Box Loss')
            axes[0, 0].set_xlabel('Epoch')
            axes[0, 0].set_ylabel('Loss')
            axes[0, 0].legend()
            axes[0, 0].grid(True)
        
        # Plot class loss
        if 'train/cls_loss' in df.columns:
            axes[0, 1].plot(df['epoch'], df['train/cls_loss'], label='Train Class Loss', color='blue')
            if 'val/cls_loss' in df.columns:
                axes[0, 1].plot(df['epoch'], df['val/cls_loss'], label='Val Class Loss', color='red')
            axes[0, 1].set_title('Class Loss')
            axes[0, 1].set_xlabel('Epoch')
            axes[0, 1].set_ylabel('Loss')
            axes[0, 1].legend()
            axes[0, 1].grid(True)
        
        # Plot mAP
        if 'metrics/mAP50(B)' in df.columns:
            axes[1, 0].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@0.5', color='green')
            if 'metrics/mAP50-95(B)' in df.columns:
                axes[1, 0].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95', color='orange')
            axes[1, 0].set_title('Mean Average Precision')
            axes[1, 0].set_xlabel('Epoch')
            axes[1, 0].set_ylabel('mAP')
            axes[1, 0].legend()
            axes[1, 0].grid(True)
        
        # Plot precision and recall
        if 'metrics/precision(B)' in df.columns:
            axes[1, 1].plot(df['epoch'], df['metrics/precision(B)'], label='Precision', color='purple')
            if 'metrics/recall(B)' in df.columns:
                axes[1, 1].plot(df['epoch'], df['metrics/recall(B)'], label='Recall', color='brown')
            axes[1, 1].set_title('Precision & Recall')
            axes[1, 1].set_xlabel('Epoch')
            axes[1, 1].set_ylabel('Score')
            axes[1, 1].legend()
            axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.suptitle('Training Results', fontsize=16, y=1.02)
        plt.show()
        
        # Print final metrics
        if len(df) > 0:
            final_row = df.iloc[-1]
            print("📊 Final Training Metrics:")
            if 'metrics/mAP50(B)' in df.columns:
                print(f"  mAP@0.5: {final_row['metrics/mAP50(B)']:.3f}")
            if 'metrics/mAP50-95(B)' in df.columns:
                print(f"  mAP@0.5:0.95: {final_row['metrics/mAP50-95(B)']:.3f}")
            if 'metrics/precision(B)' in df.columns:
                print(f"  Precision: {final_row['metrics/precision(B)']:.3f}")
            if 'metrics/recall(B)' in df.columns:
                print(f"  Recall: {final_row['metrics/recall(B)']:.3f}")
    else:
        print("❌ Training results not found")

# Plot results if training completed
try:
    if 'results' in locals() and hasattr(results, 'save_dir'):
        plot_training_results(results.save_dir)
    else:
        # Try to find the latest training run
        runs_dir = "runs/detect"
        if os.path.exists(runs_dir):
            train_dirs = [d for d in os.listdir(runs_dir) if d.startswith('train')]
            if train_dirs:
                latest_dir = os.path.join(runs_dir, sorted(train_dirs)[-1])
                plot_training_results(latest_dir)
except Exception as e:
    print(f"Could not plot training results: {e}")

## 6. Test Model on Sample Images

Load the trained model and test its detection capabilities on sample images.

In [ ]:
# Load the trained model
try:
    if os.path.exists(BEST_MODEL_PATH):
        trained_model = YOLO(BEST_MODEL_PATH)
        print(f"✅ Loaded trained model from: {BEST_MODEL_PATH}")
    else:
        print("⚠️ Trained model not found, using pretrained model")
        trained_model = YOLO(f'{MODEL_SIZE}.pt')
except Exception as e:
    print(f"❌ Error loading model: {e}")
    trained_model = YOLO(f'{MODEL_SIZE}.pt')

# Test on sample images
def test_model_on_samples(model, dataset_path, num_samples=6, confidence=0.5):
    """Test the model on sample images"""
    
    # Try validation set first, then test set
    for split in ['valid', 'test']:
        test_images_path = os.path.join(dataset_path, split, "images")
        if os.path.exists(test_images_path):
            break
    else:
        # Fallback to training images
        test_images_path = os.path.join(dataset_path, "train", "images")
    
    if not os.path.exists(test_images_path):
        print("❌ No test images found")
        return
    
    # Get sample images
    image_files = [f for f in os.listdir(test_images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    sample_files = image_files[:num_samples]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, img_file in enumerate(sample_files):
        if i >= num_samples:
            break
            
        # Load and process image
        img_path = os.path.join(test_images_path, img_file)
        image = cv2.imread(img_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Run inference
        results = model(image_rgb, conf=confidence)
        
        # Count detections
        box_count = 0
        if results[0].boxes is not None:
            box_count = len(results[0].boxes)
            
            # Draw bounding boxes
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                conf = box.conf[0].cpu().numpy()
                
                # Draw rectangle
                cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Add confidence label
                label = f"Box: {conf:.2f}"
                cv2.putText(image_rgb, label, (x1, y1-10), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Display image
        axes[i].imshow(image_rgb)
        axes[i].set_title(f"{img_file}\\nDetected: {box_count} boxes", fontsize=10)
        axes[i].axis('off')
    
    # Hide unused subplots
    for i in range(len(sample_files), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle("Model Predictions on Test Images", fontsize=16, y=1.02)
    plt.show()
    
    return len(sample_files)

# Test the model
num_tested = test_model_on_samples(trained_model, DATASET_PATH, confidence=0.5)
print(f"🔍 Tested model on {num_tested} sample images")

## 7. Implement Box Detection and Counting

Create comprehensive functions to detect boxes and implement accurate counting logic.

In [ ]:
class BoxCounter:
    """Advanced box detection and counting system"""
    
    def __init__(self, model_path, confidence=0.5, iou_threshold=0.45):
        """Initialize the box counter"""
        self.model = YOLO(model_path)
        self.confidence = confidence
        self.iou_threshold = iou_threshold
        
    def detect_boxes(self, image, return_details=False):
        """
        Detect boxes in an image
        
        Args:
            image: Input image (numpy array or path)
            return_details: Whether to return detailed detection info
            
        Returns:
            Dictionary with detection results
        """
        # Load image if path is provided
        if isinstance(image, str):
            image = cv2.imread(image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Run inference
        results = self.model(image, conf=self.confidence, iou=self.iou_threshold)
        
        # Extract detection information
        detections = {
            'count': 0,
            'boxes': [],
            'confidences': [],
            'areas': [],
            'centers': []
        }
        
        if results[0].boxes is not None:
            boxes = results[0].boxes
            detections['count'] = len(boxes)
            
            for box in boxes:
                # Get box coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                conf = float(box.conf[0].cpu().numpy())
                
                # Calculate additional metrics
                area = (x2 - x1) * (y2 - y1)
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                
                detections['boxes'].append([x1, y1, x2, y2])
                detections['confidences'].append(conf)
                detections['areas'].append(area)
                detections['centers'].append([center_x, center_y])
        
        if return_details:
            return detections
        else:
            return detections['count']
    
    def visualize_detections(self, image, detections=None, save_path=None):
        """
        Visualize box detections on image
        
        Args:
            image: Input image
            detections: Detection results (optional, will compute if not provided)
            save_path: Path to save visualization
            
        Returns:
            Annotated image
        """
        # Load image if path is provided
        if isinstance(image, str):
            image_path = image
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Get detections if not provided
        if detections is None:
            detections = self.detect_boxes(image, return_details=True)
        
        # Create a copy for annotation
        annotated_image = image.copy()
        
        # Draw bounding boxes
        for i, (box, conf, area, center) in enumerate(zip(
            detections['boxes'], 
            detections['confidences'], 
            detections['areas'],
            detections['centers']
        )):
            x1, y1, x2, y2 = box
            
            # Color based on confidence (green for high, yellow for medium, red for low)
            if conf > 0.7:
                color = (0, 255, 0)  # Green
            elif conf > 0.5:
                color = (255, 255, 0)  # Yellow
            else:
                color = (255, 0, 0)  # Red
            
            # Draw rectangle
            cv2.rectangle(annotated_image, (x1, y1), (x2, y2), color, 2)
            
            # Draw center point
            cv2.circle(annotated_image, tuple(center), 3, color, -1)
            
            # Add label with confidence and box number
            label = f"Box {i+1}: {conf:.2f}"
            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            cv2.rectangle(annotated_image, (x1, y1-label_size[1]-10), 
                         (x1+label_size[0], y1), color, -1)
            cv2.putText(annotated_image, label, (x1, y1-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        
        # Add summary text
        summary_text = f"Total Boxes: {detections['count']}"
        if detections['confidences']:
            avg_conf = np.mean(detections['confidences'])
            summary_text += f" | Avg Confidence: {avg_conf:.3f}"
        
        cv2.putText(annotated_image, summary_text, (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
        cv2.putText(annotated_image, summary_text, (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        
        # Save if path provided
        if save_path:
            cv2.imwrite(save_path, cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
        
        return annotated_image
    
    def batch_count(self, image_paths, save_results=True, output_dir="results"):
        """
        Count boxes in multiple images
        
        Args:
            image_paths: List of image paths or directory path
            save_results: Whether to save results
            output_dir: Directory to save results
            
        Returns:
            List of results for each image
        """
        # Handle directory input
        if isinstance(image_paths, str) and os.path.isdir(image_paths):
            image_dir = image_paths
            image_paths = []
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                image_paths.extend(Path(image_dir).glob(f"*{ext}"))
                image_paths.extend(Path(image_dir).glob(f"*{ext.upper()}"))
        
        results = []
        
        # Create output directory
        if save_results:
            os.makedirs(output_dir, exist_ok=True)
        
        for i, img_path in enumerate(image_paths):
            try:
                print(f"Processing {i+1}/{len(image_paths)}: {Path(img_path).name}")
                
                # Detect boxes
                detections = self.detect_boxes(str(img_path), return_details=True)
                
                # Create result record
                result = {
                    'image_path': str(img_path),
                    'image_name': Path(img_path).name,
                    'box_count': detections['count'],
                    'avg_confidence': np.mean(detections['confidences']) if detections['confidences'] else 0,
                    'max_confidence': max(detections['confidences']) if detections['confidences'] else 0,
                    'min_confidence': min(detections['confidences']) if detections['confidences'] else 0,
                    'total_area': sum(detections['areas']),
                    'avg_box_area': np.mean(detections['areas']) if detections['areas'] else 0
                }\n                results.append(result)\n                \n                # Save visualization if requested\n                if save_results:\n                    img = cv2.imread(str(img_path))\n                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n                    annotated = self.visualize_detections(img_rgb, detections)\n                    \n                    output_name = f"{Path(img_path).stem}_detected.jpg"\n                    output_path = os.path.join(output_dir, output_name)\n                    cv2.imwrite(output_path, cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))\n                \n            except Exception as e:\n                print(f"Error processing {img_path}: {e}")\n                result = {\n                    'image_path': str(img_path),\n                    'image_name': Path(img_path).name,\n                    'box_count': 0,\n                    'error': str(e)\n                }\n                results.append(result)\n        \n        # Save summary results\n        if save_results and results:\n            summary_path = os.path.join(output_dir, "counting_results.json")\n            with open(summary_path, 'w') as f:\n                json.dump(results, f, indent=2)\n            \n            # Create CSV summary\n            df = pd.DataFrame(results)\n            csv_path = os.path.join(output_dir, "counting_summary.csv")\n            df.to_csv(csv_path, index=False)\n            \n            print(f"✅ Results saved to {output_dir}")\n        \n        return results\n\n# Initialize the box counter with the trained model\ntry:\n    box_counter = BoxCounter(BEST_MODEL_PATH, confidence=0.5, iou_threshold=0.45)\n    print("✅ Box counter initialized with trained model")\nexcept:\n    box_counter = BoxCounter(f'{MODEL_SIZE}.pt', confidence=0.5, iou_threshold=0.45)\n    print("⚠️ Using pretrained model for box counter")

In [ ]:
# Test the box counter on sample images
def demo_box_counting(box_counter, dataset_path, num_samples=4):
    """Demonstrate box counting functionality"""
    
    # Get test images
    test_images_path = os.path.join(dataset_path, "valid", "images")
    if not os.path.exists(test_images_path):
        test_images_path = os.path.join(dataset_path, "train", "images")
    
    if not os.path.exists(test_images_path):
        print("❌ No test images found")
        return
    
    image_files = [f for f in os.listdir(test_images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    sample_files = image_files[:num_samples]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    total_boxes = 0
    
    for i, img_file in enumerate(sample_files):
        img_path = os.path.join(test_images_path, img_file)
        
        # Load image
        image = cv2.imread(img_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Count boxes
        detections = box_counter.detect_boxes(image_rgb, return_details=True)
        count = detections['count']
        total_boxes += count
        
        # Visualize
        annotated = box_counter.visualize_detections(image_rgb, detections)
        
        # Display
        axes[i].imshow(annotated)
        axes[i].set_title(f"{img_file}\\nBoxes: {count}", fontsize=12)
        axes[i].axis('off')
        
        # Print details
        print(f"📦 {img_file}: {count} boxes detected")
        if detections['confidences']:
            print(f"   Confidence range: {min(detections['confidences']):.3f} - {max(detections['confidences']):.3f}")
    
    # Hide unused subplots
    for i in range(len(sample_files), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"Box Counting Demo - Total: {total_boxes} boxes", fontsize=16, y=1.02)
    plt.show()
    
    return total_boxes

# Run the demo
total_detected = demo_box_counting(box_counter, DATASET_PATH)
print(f"\\n🎯 Demo completed! Total boxes detected: {total_detected}")

## 8. Evaluate Model Performance

Calculate comprehensive performance metrics including precision, recall, mAP, and counting accuracy.

In [ ]:
# Evaluate model performance on validation set
def evaluate_model_performance(model, dataset_path):
    """Comprehensive model evaluation"""
    
    # Run validation using YOLO's built-in validation
    print("🔍 Running model validation...")
    
    try:\n        # Validate on the dataset\n        val_results = model.val(data=os.path.join(dataset_path, "data.yaml"))\n        \n        print("📊 Validation Metrics:")\n        metrics = val_results.results_dict\n        \n        # Print key metrics\n        if 'metrics/mAP50(B)' in metrics:\n            print(f"  mAP@0.5: {metrics['metrics/mAP50(B)']:.4f}")\n        if 'metrics/mAP50-95(B)' in metrics:\n            print(f"  mAP@0.5:0.95: {metrics['metrics/mAP50-95(B)']:.4f}")\n        if 'metrics/precision(B)' in metrics:\n            print(f"  Precision: {metrics['metrics/precision(B)']:.4f}")\n        if 'metrics/recall(B)' in metrics:\n            print(f"  Recall: {metrics['metrics/recall(B)']:.4f}")\n        \n        return val_results\n        \n    except Exception as e:\n        print(f"❌ Validation failed: {e}")\n        return None\n\n# Custom counting accuracy evaluation\ndef evaluate_counting_accuracy(box_counter, dataset_path, split='valid'):\n    """Evaluate counting accuracy against ground truth"""\n    \n    images_path = os.path.join(dataset_path, split, "images")\n    labels_path = os.path.join(dataset_path, split, "labels")\n    \n    if not os.path.exists(images_path) or not os.path.exists(labels_path):\n        print(f"❌ {split} set not found")\n        return None\n    \n    image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]\n    \n    results = []\n    absolute_errors = []\n    relative_errors = []\n    \n    print(f"📊 Evaluating counting accuracy on {len(image_files)} images...")\n    \n    for img_file in image_files:\n        # Load image\n        img_path = os.path.join(images_path, img_file)\n        \n        # Get predicted count\n        predicted_count = box_counter.detect_boxes(img_path)\n        \n        # Get ground truth count\n        label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt')\n        label_path = os.path.join(labels_path, label_file)\n        \n        ground_truth_count = 0\n        if os.path.exists(label_path):\n            with open(label_path, 'r') as f:\n                ground_truth_count = len(f.readlines())\n        \n        # Calculate errors\n        absolute_error = abs(predicted_count - ground_truth_count)\n        relative_error = absolute_error / max(ground_truth_count, 1) * 100\n        \n        results.append({\n            'image': img_file,\n            'predicted': predicted_count,\n            'ground_truth': ground_truth_count,\n            'absolute_error': absolute_error,\n            'relative_error': relative_error\n        })\n        \n        absolute_errors.append(absolute_error)\n        relative_errors.append(relative_error)\n    \n    # Calculate summary statistics\n    mean_absolute_error = np.mean(absolute_errors)\n    mean_relative_error = np.mean(relative_errors)\n    perfect_matches = sum(1 for e in absolute_errors if e == 0)\n    accuracy = perfect_matches / len(results) * 100\n    \n    print(f"\\n📈 Counting Accuracy Results:")\n    print(f"  Perfect matches: {perfect_matches}/{len(results)} ({accuracy:.1f}%)")\n    print(f"  Mean Absolute Error: {mean_absolute_error:.2f}")\n    print(f"  Mean Relative Error: {mean_relative_error:.1f}%")\n    print(f"  Max Absolute Error: {max(absolute_errors)}")\n    print(f"  Min Absolute Error: {min(absolute_errors)}")\n    \n    return {\n        'results': results,\n        'perfect_matches': perfect_matches,\n        'accuracy': accuracy,\n        'mean_absolute_error': mean_absolute_error,\n        'mean_relative_error': mean_relative_error\n    }\n\n# Run evaluations\nprint("🚀 Starting model evaluation...")\n\n# Standard YOLO validation\nval_results = evaluate_model_performance(trained_model, DATASET_PATH)\n\n# Custom counting accuracy evaluation\ncounting_results = evaluate_counting_accuracy(box_counter, DATASET_PATH)

In [ ]:
# Visualize evaluation results
def plot_evaluation_results(counting_results):\n    """Plot evaluation metrics and error distribution"""\n    \n    if not counting_results:\n        print("❌ No counting results to plot")\n        return\n    \n    results = counting_results['results']\n    \n    # Create subplots\n    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n    \n    # 1. Predicted vs Ground Truth\n    predicted = [r['predicted'] for r in results]\n    ground_truth = [r['ground_truth'] for r in results]\n    \n    axes[0, 0].scatter(ground_truth, predicted, alpha=0.6)\n    axes[0, 0].plot([0, max(ground_truth + predicted)], [0, max(ground_truth + predicted)], 'r--', label='Perfect prediction')\n    axes[0, 0].set_xlabel('Ground Truth Count')\n    axes[0, 0].set_ylabel('Predicted Count')\n    axes[0, 0].set_title('Predicted vs Ground Truth')\n    axes[0, 0].legend()\n    axes[0, 0].grid(True)\n    \n    # 2. Absolute Error Distribution\n    absolute_errors = [r['absolute_error'] for r in results]\n    axes[0, 1].hist(absolute_errors, bins=20, alpha=0.7, color='orange')\n    axes[0, 1].set_xlabel('Absolute Error')\n    axes[0, 1].set_ylabel('Frequency')\n    axes[0, 1].set_title('Distribution of Absolute Errors')\n    axes[0, 1].grid(True)\n    \n    # 3. Error vs Ground Truth Count\n    axes[1, 0].scatter(ground_truth, absolute_errors, alpha=0.6, color='red')\n    axes[1, 0].set_xlabel('Ground Truth Count')\n    axes[1, 0].set_ylabel('Absolute Error')\n    axes[1, 0].set_title('Error vs Ground Truth Count')\n    axes[1, 0].grid(True)\n    \n    # 4. Summary Statistics\n    stats_text = f\"\"\"\nPerfect Matches: {counting_results['perfect_matches']}/{len(results)} ({counting_results['accuracy']:.1f}%)\nMean Absolute Error: {counting_results['mean_absolute_error']:.2f}\nMean Relative Error: {counting_results['mean_relative_error']:.1f}%\nMax Error: {max(absolute_errors)}\nMin Error: {min(absolute_errors)}\n\"\"\"\n    \n    axes[1, 1].text(0.1, 0.5, stats_text, transform=axes[1, 1].transAxes, \n                   fontsize=12, verticalalignment='center',\n                   bbox=dict(boxstyle="round", facecolor='lightblue'))\n    axes[1, 1].set_title('Summary Statistics')\n    axes[1, 1].axis('off')\n    \n    plt.tight_layout()\n    plt.suptitle('Model Evaluation Results', fontsize=16, y=1.02)\n    plt.show()\n\n# Plot evaluation results\nif counting_results:\n    plot_evaluation_results(counting_results)\nelse:\n    print("⚠️ No counting results available for plotting")

## 9. Deploy Model for Real-time Box Counting

Implement real-time inference pipeline for processing video streams or batch images.

In [ ]:
# Save the trained model for deployment\ndef save_deployment_model(model_path, output_dir="../models"):\n    """Save model and configuration for deployment"""\n    \n    os.makedirs(output_dir, exist_ok=True)\n    \n    # Copy best model\n    if os.path.exists(model_path):\n        deployment_model_path = os.path.join(output_dir, "box_detection_model.pt")\n        import shutil\n        shutil.copy2(model_path, deployment_model_path)\n        print(f"✅ Model saved to: {deployment_model_path}")\n        \n        # Save model configuration\n        config = {\n            'model_path': 'box_detection_model.pt',\n            'model_type': MODEL_SIZE,\n            'num_classes': NUM_CLASSES,\n            'class_names': CLASS_NAMES,\n            'confidence_threshold': 0.5,\n            'iou_threshold': 0.45,\n            'image_size': 640\n        }\n        \n        config_path = os.path.join(output_dir, "model_config.json")\n        with open(config_path, 'w') as f:\n            json.dump(config, f, indent=2)\n        \n        print(f"✅ Configuration saved to: {config_path}")\n        return deployment_model_path\n    else:\n        print(f"❌ Model not found at: {model_path}")\n        return None\n\n# Real-time inference class\nclass RealTimeBoxCounter:\n    """Real-time box counting system"""\n    \n    def __init__(self, model_path, confidence=0.5):\n        self.model = YOLO(model_path)\n        self.confidence = confidence\n        self.frame_count = 0\n        self.total_detections = 0\n        \n    def process_video(self, video_path, output_path=None, display=True):\n        """Process video file for box counting"""\n        \n        cap = cv2.VideoCapture(video_path)\n        \n        if not cap.isOpened():\n            print(f"❌ Cannot open video: {video_path}")\n            return\n        \n        # Get video properties\n        fps = int(cap.get(cv2.CAP_PROP_FPS))\n        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))\n        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))\n        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))\n        \n        print(f"📺 Processing video: {width}x{height} @ {fps}fps, {total_frames} frames")\n        \n        # Setup video writer if output path provided\n        out = None\n        if output_path:\n            fourcc = cv2.VideoWriter_fourcc(*'mp4v')\n            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))\n        \n        frame_results = []\n        \n        try:\n            while True:\n                ret, frame = cap.read()\n                if not ret:\n                    break\n                \n                self.frame_count += 1\n                \n                # Convert to RGB for YOLO\n                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n                \n                # Run detection\n                results = self.model(frame_rgb, conf=self.confidence)\n                \n                # Count boxes\n                box_count = 0\n                if results[0].boxes is not None:\n                    box_count = len(results[0].boxes)\n                    \n                    # Draw bounding boxes\n                    for box in results[0].boxes:\n                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)\n                        conf = box.conf[0].cpu().numpy()\n                        \n                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)\n                        cv2.putText(frame, f'{conf:.2f}', (x1, y1-10), \n                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)\n                \n                self.total_detections += box_count\n                \n                # Add frame info\n                info_text = f"Frame: {self.frame_count}/{total_frames} | Boxes: {box_count}"\n                cv2.putText(frame, info_text, (10, 30), \n                          cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)\n                \n                frame_results.append({\n                    'frame': self.frame_count,\n                    'box_count': box_count\n                })\n                \n                # Write frame if output specified\n                if out:\n                    out.write(frame)\n                \n                # Display frame\n                if display:\n                    cv2.imshow('Box Detection', frame)\n                    if cv2.waitKey(1) & 0xFF == ord('q'):\n                        break\n                \n                # Progress update\n                if self.frame_count % 30 == 0:\n                    progress = (self.frame_count / total_frames) * 100\n                    print(f"Progress: {progress:.1f}% - Avg boxes/frame: {self.total_detections/self.frame_count:.1f}")\n        \n        finally:\n            cap.release()\n            if out:\n                out.release()\n            if display:\n                cv2.destroyAllWindows()\n        \n        print(f"✅ Video processing complete!")\n        print(f"📊 Total frames: {self.frame_count}")\n        print(f"📦 Total detections: {self.total_detections}")\n        print(f"📈 Average boxes per frame: {self.total_detections/self.frame_count:.2f}")\n        \n        return frame_results\n    \n    def process_webcam(self, camera_id=0):\n        """Process live webcam feed"""\n        \n        cap = cv2.VideoCapture(camera_id)\n        \n        if not cap.isOpened():\n            print(f"❌ Cannot open camera {camera_id}")\n            return\n        \n        print("📷 Starting webcam feed. Press 'q' to quit.")\n        \n        try:\n            while True:\n                ret, frame = cap.read()\n                if not ret:\n                    break\n                \n                # Convert to RGB for YOLO\n                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n                \n                # Run detection\n                results = self.model(frame_rgb, conf=self.confidence)\n                \n                # Count and draw boxes\n                box_count = 0\n                if results[0].boxes is not None:\n                    box_count = len(results[0].boxes)\n                    \n                    for box in results[0].boxes:\n                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)\n                        conf = box.conf[0].cpu().numpy()\n                        \n                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)\n                        cv2.putText(frame, f'Box: {conf:.2f}', (x1, y1-10), \n                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)\n                \n                # Add info text\n                cv2.putText(frame, f'Boxes Detected: {box_count}', (10, 30), \n                          cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)\n                cv2.putText(frame, \"Press 'q' to quit\", (10, frame.shape[0] - 20), \n                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)\n                \n                # Display frame\n                cv2.imshow('Real-time Box Detection', frame)\n                \n                if cv2.waitKey(1) & 0xFF == ord('q'):\n                    break\n        \n        finally:\n            cap.release()\n            cv2.destroyAllWindows()\n\n# Save the deployment model\ndeployment_model_path = save_deployment_model(BEST_MODEL_PATH)\n\n# Create deployment instructions\ndeployment_instructions = f\"\"\"\n# 🚀 Deployment Instructions\n\n## Model Files\n- **Model**: `{deployment_model_path or 'box_detection_model.pt'}`\n- **Config**: `../models/model_config.json`\n\n## Usage Examples\n\n### 1. Single Image Processing\n```python\nfrom ultralytics import YOLO\n\n# Load model\nmodel = YOLO('../models/box_detection_model.pt')\n\n# Detect boxes\nresults = model('image.jpg', conf=0.5)\nbox_count = len(results[0].boxes) if results[0].boxes else 0\nprint(f\"Detected {{box_count}} boxes\")\n```\n\n### 2. Batch Processing\n```python\n# Process multiple images\nresults = model(['image1.jpg', 'image2.jpg'], conf=0.5)\nfor i, result in enumerate(results):\n    count = len(result.boxes) if result.boxes else 0\n    print(f\"Image {{i+1}}: {{count}} boxes\")\n```\n\n### 3. Real-time Video Processing\n```python\n# Initialize real-time counter\ncounter = RealTimeBoxCounter('../models/box_detection_model.pt')\n\n# Process video file\ncounter.process_video('input_video.mp4', 'output_video.mp4')\n\n# Or process webcam feed\ncounter.process_webcam(0)  # Use camera 0\n```\n\n## Integration with Web Application\nThe trained model can be integrated with the Streamlit web application:\n\n1. Copy the model file to the `models/` directory\n2. Update the `MODEL_PATH` in `.env` or `config.yaml`\n3. Run the web application: `streamlit run app.py`\n\n## API Integration\nFor production deployment, consider:\n- FastAPI for REST API endpoints\n- Docker containerization\n- GPU acceleration setup\n- Model optimization (TensorRT, ONNX)\n\"\"\"\n\nprint(deployment_instructions)\n\n# Save deployment instructions\nwith open('../DEPLOYMENT.md', 'w') as f:\n    f.write(deployment_instructions)\n\nprint(\"\\n✅ Deployment files and instructions created!\")\nprint(\"📄 See DEPLOYMENT.md for detailed usage instructions\")